In [16]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import cv2
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch.nn.functional as F

In [17]:
class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.ReLU(True),
            nn.Conv2d(channels // reduction, channels, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = self.sigmoid(avg_out + max_out)
        return x * out

In [18]:
class SpatialAttention(nn.Module):
    def __init__(self):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=7, padding=3)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(out)
        return x * self.sigmoid(out)

In [19]:
class RCAB(nn.Module):
    def __init__(self, channels):
        super(RCAB, self).__init__()
        self.body = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(channels, channels, 3, padding=1),
            ChannelAttention(channels),
            SpatialAttention()
        )

    def forward(self, x):
        return x + self.body(x)


In [20]:
class EnhancedModel(nn.Module):
    def __init__(self, num_blocks=16):
        super(EnhancedModel, self).__init__()
        
        # Initial feature extraction
        self.init_feature = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(True)
        )
        
        # Denoising module
        self.denoise = nn.Sequential(
            *[RCAB(64) for _ in range(4)]
        )
        
        # Deep feature extraction
        self.feature_blocks = nn.Sequential(
            *[RCAB(64) for _ in range(num_blocks)]
        )
        
        # Upsampling module
        self.upsampling = nn.Sequential(
            nn.Conv2d(64, 256, kernel_size=3, padding=1),
            nn.PixelShuffle(2),
            nn.ReLU(True),
            RCAB(64),
            nn.Conv2d(64, 256, kernel_size=3, padding=1),
            nn.PixelShuffle(2),
            nn.ReLU(True),
            RCAB(64)
        )
        
        # Reconstruction
        self.reconstruction = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(64, 3, kernel_size=3, padding=1)
        )

    def forward(self, x):
        # Initial feature extraction
        init_features = self.init_feature(x)
        
        # Denoising
        denoised = self.denoise(init_features)
        
        # Deep feature extraction
        features = self.feature_blocks(denoised)
        features += init_features
        
        # Upsampling
        upsampled = self.upsampling(features)
        
        # Final reconstruction
        out = self.reconstruction(upsampled)
        return torch.clamp(out, 0, 1)



In [21]:
class EnhancedDataset(Dataset):
    def __init__(self, noise_path, gt_path=None, is_test=False, patch_size=128):
        self.noise_path = noise_path
        self.gt_path = gt_path
        self.is_test = is_test
        self.patch_size = patch_size
        self.image_files = sorted([f for f in os.listdir(noise_path) if f.endswith('.png')])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        noise_img_path = os.path.join(self.noise_path, img_name)
        
        # Check if noisy image exists
        if not os.path.exists(noise_img_path):
            raise FileNotFoundError(f"Noisy image not found: {noise_img_path}")
        
        noise_image = cv2.imread(noise_img_path)
        if noise_image is None:
            raise ValueError(f"Failed to load noisy image: {noise_img_path}")
        noise_image = cv2.cvtColor(noise_image, cv2.COLOR_BGR2RGB)
        
        if not self.is_test and self.gt_path:
            # Handle ground truth image name
            gt_name = img_name
            if 'test_' in img_name:
                gt_name = img_name.replace('test_', 'gt_')
            elif 'val_' in img_name:
                gt_name = img_name.replace('val_', 'gt_')
            elif not img_name.startswith('gt_'):
                gt_name = f'gt_{img_name}'
                
            gt_img_path = os.path.join(self.gt_path, gt_name)
            
            # Check if ground truth image exists
            if not os.path.exists(gt_img_path):
                raise FileNotFoundError(f"Ground truth image not found: {gt_img_path}")
            
            gt_image = cv2.imread(gt_img_path)
            if gt_image is None:
                raise ValueError(f"Failed to load ground truth image: {gt_img_path}")
            gt_image = cv2.cvtColor(gt_image, cv2.COLOR_BGR2RGB)
            
            # Rest of the processing remains the same
            if not self.is_test:
                h, w = gt_image.shape[:2]
                h = h - (h % 4)
                w = w - (w % 4)
                gt_image = gt_image[:h, :w]
                noise_image = cv2.resize(noise_image, (w//4, h//4), interpolation=cv2.INTER_CUBIC)
                
                if self.patch_size:
                    lr_patch_size = self.patch_size // 4
                    hr_patch_size = self.patch_size
                    
                    lr_h, lr_w = noise_image.shape[:2]
                    lr_x = np.random.randint(0, max(1, lr_w - lr_patch_size + 1))
                    lr_y = np.random.randint(0, max(1, lr_h - lr_patch_size + 1))
                    
                    hr_x, hr_y = lr_x * 4, lr_y * 4
                    
                    noise_image = noise_image[lr_y:lr_y + lr_patch_size, lr_x:lr_x + lr_patch_size]
                    gt_image = gt_image[hr_y:hr_y + hr_patch_size, hr_x:hr_x + hr_patch_size]
            
            noise_image = noise_image.astype(np.float32) / 255.0
            gt_image = gt_image.astype(np.float32) / 255.0
            
            return {
                'noise': torch.from_numpy(noise_image).permute(2, 0, 1),
                'gt': torch.from_numpy(gt_image).permute(2, 0, 1),
                'name': img_name
            }
        
        # For test data
        h, w = noise_image.shape[:2]
        h = h - (h % 4)
        w = w - (w % 4)
        noise_image = noise_image[:h, :w]
        noise_image = noise_image.astype(np.float32) / 255.0
        
        return {
            'noise': torch.from_numpy(noise_image).permute(2, 0, 1),
            'name': img_name
        }

In [22]:

def calculate_psnr(pred, target):
    mse = torch.mean((pred - target) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * torch.log10(1.0 / torch.sqrt(mse))

In [23]:
def create_submission(model, test_loader, device):
    output_path = '/kaggle/working/outputs/predicted_images'
    os.makedirs(output_path, exist_ok=True)
    
    model.eval()
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Generating predictions"):
            noise_imgs = batch['noise'].to(device)
            outputs = model(noise_imgs)
            
            for i, img_name in enumerate(batch['name']):
                output = outputs[i].cpu().numpy().transpose(1, 2, 0)
                output = (output * 255).astype(np.uint8)
                output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
                cv2.imwrite(os.path.join(output_path, img_name), output)
    
    # Create submission CSV
    images_to_csv(output_path, 'submission.csv')




In [24]:
def train_model(model, train_loader, val_loader, test_loader, num_epochs, device):
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)
    
    best_psnr = 0
    best_model_state = None  # Store best model state
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            noise_imgs = batch['noise'].to(device)
            gt_imgs = batch['gt'].to(device)
            
            optimizer.zero_grad()
            outputs = model(noise_imgs)
            loss = criterion(outputs, gt_imgs)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_psnr = 0
        with torch.no_grad():
            for batch in val_loader:
                noise_imgs = batch['noise'].to(device)
                gt_imgs = batch['gt'].to(device)
                outputs = model(noise_imgs)
                val_psnr += calculate_psnr(outputs, gt_imgs)
        
        val_psnr /= len(val_loader)
        if val_psnr > best_psnr:
            best_psnr = val_psnr
            best_model_state = model.state_dict().copy()  # Save best state
            torch.save(best_model_state, 'best_model.pth')
        
        scheduler.step()
        print(f'Epoch {epoch+1}, Loss: {train_loss/len(train_loader):.4f}, PSNR: {val_psnr:.2f}')
    
    # After training, load best model and create submission
    print("Training completed. Creating submission with best model...")
    model.load_state_dict(best_model_state)
    create_submission(model, test_loader, device)

In [25]:
def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L')
            image_array = np.array(image).flatten()[::8]
            image_id = filename.split('.')[0].replace('test_', 'gt_')
            data_rows.append([image_id, *image_array])
    
    columns = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=columns)
    df.to_csv(output_csv, index=False)

In [26]:
def main():
    # Paths
    noise_train_path = r'/kaggle/input/dlp-jan-2025-nppe-3/archive/train/train'
    gt_train_path = r'/kaggle/input/dlp-jan-2025-nppe-3/archive/train/gt'
    noise_val_path = r'/kaggle/input/dlp-jan-2025-nppe-3/archive/val/val'
    gt_val_path = r'/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt'
    noise_test_path = r'/kaggle/input/dlp-jan-2025-nppe-3/archive/test'

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Create datasets
    train_dataset = EnhancedDataset(noise_train_path, gt_train_path)
    val_dataset = EnhancedDataset(noise_val_path, gt_val_path)
    test_dataset = EnhancedDataset(noise_test_path, is_test=True)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    # Create and train model with test_loader
    model = EnhancedModel().to(device)
    train_model(model, train_loader, val_loader, test_loader, num_epochs=100, device=device)
    

In [27]:
if __name__ == "__main__":
    main()

Epoch 1/100: 100%|██████████| 139/139 [00:12<00:00, 11.02it/s]


Epoch 1, Loss: 0.0403, PSNR: 33.61


Epoch 2/100: 100%|██████████| 139/139 [00:12<00:00, 11.15it/s]


Epoch 2, Loss: 0.0159, PSNR: 35.96


Epoch 3/100: 100%|██████████| 139/139 [00:12<00:00, 11.13it/s]


...

Epoch 99/100: 100%|██████████| 139/139 [00:12<00:00, 10.98it/s]


Epoch 99, Loss: 0.0098, PSNR: 40.08


Epoch 100/100: 100%|██████████| 139/139 [00:12<00:00, 11.04it/s]


Epoch 100, Loss: 0.0099, PSNR: 39.37
Training completed. Creating submission with best model...


Generating predictions: 100%|██████████| 60/60 [00:13<00:00,  4.44it/s]
